In [ ]:
from sketch.examples.prompt_machine import *

async def chatbot(response_callback):
    messages = []
    async with websockets.connect(PM_SETTINGS['uri'], extra_headers={"Cookie": f"Authorization=Bearer {PM_SETTINGS['BOT_TOKEN']}"}) as ws:
        while True:
            try:
                raw_data = await asyncio.wait_for(ws.recv(), timeout=0.5)
                data = json.loads(raw_data)
                print("log:", data)
                if data.get("meta", False):
                    continue
                if data.get("replay", False) or data["sender"] == "datAI":
                    messages.append(data)
                    continue
                if data["message"] == "sudo clear":
                    print("Sudo clearing")
                    await ws.send(json.dumps({"message": "sudo clear thread"}))
                    messages = []
                    continue
                if data["message"] == "break":
                    break
                messages.append(data)
            except asyncio.TimeoutError:
                # not sure why I'm doing this, it felt important to have an "infinite" loop...
                continue
            await ws.send(json.dumps({"typing": True}))
            response = await response_callback(ws, "\n".join([f"{m['sender']}: {m['message']}" for m in messages[-1:]]))
            await ws.send(json.dumps({"message": response}))
            await ws.send(json.dumps({"typing": False}))


In [ ]:
sqlExecutor = GPT3Prompt(
    "sqlExecutor",
    """
The user's intent in this conversation {{ userIntent }}.
Answer the users intent with SQLite SQL from the following database.
|| Database Context ||
{{ database_context }}
===
Query to answer [{{ userIntent }}]:
{% if previousAttempt %}
DIFFERING ATTEMPTS: (3)
(1)
{{ previousAttempt }}

(2)
{% endif %}
```""",
    stop="```",
    model_name="code-davinci-002"
)

In [ ]:
composeDataDrivenAnswer = GPT3Prompt(
    "composeDataDrivenAnswer",
    """"
===
The query and response:
{{ sql }}
Result:
{{ result }}
===
what the query does {{ whatWasAnswered }}
===
What should datAI say in response to the last message from the user, answering with the data returned from the query.
(Note: AI can say something funny too!)

datAI:""",
)

In [ ]:
whatQuestionWasAnswered = GPT3Prompt(
    "whatQuestionWasAnswered",
    """"
The query and response:
{{ sql }}
Result:
{{ result }}
===
Explain the query that was run to a 5 year old
datAI:""",
)

In [ ]:
database_context = get_db_context()

async def simple_qa_bot(ws, conversation):
    userIntent = getUserIntent(conversation=conversation)
    await ws.send(json.dumps({"message": "I am interpreting your intent as... {}".format(userIntent)}))
    trials = 0
    previousAttempt = ""
    previousPreviousAttempt = ""
    while trials < 8:
        sqlresponse = sqlExecutor(
            userIntent=userIntent,
            previousAttempt=previousAttempt,
            database_context=database_context,
        )
        res = exec_sql(sqlresponse)
        validSQL = gotAValidSQLResponse(
            conversation=conversation, sql=sqlresponse, result=res
        )
        if starts_with_y(validSQL):
            whatWasAnswered = whatQuestionWasAnswered(
                sql=sqlresponse, result=res
            )
            await ws.send(json.dumps({"message": f"The question that was answered is: {whatWasAnswered}"}))
            return composeDataDrivenAnswer(
                whatWasAnswered=whatWasAnswered, sql=sqlresponse, result=res
            )
        trials += 1
        previousPreviousAttempt = previousAttempt
        previousAttempt = f"```{sqlresponse}```\nResult:{res[:100]}\n"
        if previousAttempt == previousPreviousAttempt:
            previousAttempt = f"STOP DOING THE SAME THING!!!!! TRY HARDER {userIntent} {userIntent} {userIntent}"
        await ws.send(json.dumps({"message": "My previous attempt was... but it clearly failed, trying again" + previousAttempt}))
    return f"I'm sorry, I tried to execute {previousAttempt} but I couldn't get a valid response. Please try again."

In [ ]:
await chatbot(simple_qa_bot)

In [ ]:
# SELECT player_game_log.game_id, SUM(player_game_log.pts) as total_points
# FROM player_game_log
# LEFT JOIN game ON game.game_id = player_game_log.game_id
# WHERE game.season_id = 2021
# GROUP BY game.game_id
# HAVING total_points > 0
# ORDER BY total_points DESC
# LIMIT 3;

In [ ]:
execute_sql(
    """
    SELECT pgl1.game_id, player1.player_name AS player1, player2.player_name AS player2 
    FROM (SELECT * FROM player_game_log WHERE pts > 40) AS pgl1 
    LEFT JOIN (SELECT * FROM player_game_log WHERE pts > 40) AS pgl2 ON pgl1.game_id = pgl2.game_id AND pgl1.team_id = pgl2.team_id AND pgl1.player_id != pgl2.player_id 
    LEFT JOIN player as player1 ON player1.player_id = pgl1.player_id 
    LEFT JOIN player as player2 ON player2.player_id = pgl2.player_id
    """
)



In [ ]:
execute_sql(
    """
    SELECT pgl1.game_id, pgl1.player_id, pgl2.player_id
    FROM (SELECT * FROM player_game_log WHERE pts > 40) AS pgl1 
    LEFT JOIN (SELECT * FROM player_game_log WHERE pts > 40) AS pgl2 ON pgl1.game_id = pgl2.game_id AND pgl1.team_id = pgl2.team_id AND pgl1.player_id != pgl2.player_id 
    """)